<a href="https://colab.research.google.com/github/wyuinche/melon_arena/blob/master/token_tag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**아래건 카이 때문입니다 안해도 될거 같아유**

In [ ]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python


우리의 필요한 라이브러리들 입니다.

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import json
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
import re
from typing import *
#from khaiii import KhaiiiApi  ##요건 khaiii쓰면 해주세요
from tqdm import tqdm
from time import sleep

In [ ]:
#구글 드라이브 아니면 지워주세요 :)
from google.colab import drive
drive.mount('/gdrive', force_remount=True)


In [ ]:
# data_path = your_path :)
token = pd.read_json('/gdrive/My Drive/Colab Notebooks/token_final.json', orient = 'table')
test = pd.read_json('/gdrive/My Drive/Colab Notebooks/test_token.json', orient='table' ) #얘는 그 제가 kahiii썼다는 가정하에 token만들어서 test에 붙여놓은거에요
val = pd.read_json('/gdrive/My Drive/Colab Notebooks/val_token.json', orient='table')
#test = pd.read_json('/gdrive/My Drive/Colab Notebooks/test.json', typ='frame', encoding='utf-8')
#val = pd.read_json('/gdrive/My Drive/Colab Notebooks/val.json', typ='frame', encoding='utf-8')

In [ ]:
#이건 빈 게 뭔지 알라구 하는 거:)
#if len(test['plylst_title'][0])==0:
  #print('yes')

In [ ]:
#이것두 그냥 하고 싶어서임 ㅇㅇ
#if len(test['plylst_title'][5]) != 0:
#  print('yes')

#이건 토큰화 하는 함수입니다. 토큰화 되면 안해도 되요

In [ ]:
tokenizer = KhaiiiApi()
#플레이리스트에서 공백은 일단 채워둘게요
def re_sub(series: pd.Series) -> pd.Series:
    series = series.str.replace(pat=r'[ㄱ-ㅎ]', repl=r'', regex=True)  # ㅋ 제거용
    series = series.str.replace(pat=r'[^\w\s]', repl=r'', regex=True)  # 특수문자 제거
    series = series.str.replace(pat=r'[\u3000]+', repl=r'', regex=True)  # u3000 제거
    return series

#토큰 가져오기
def get_token(title: str, tokenizer)-> List[List]:
    
    if len(title)== 0 or title== ' ':  # 제목이 공백인 경우 tokenizer에러 발생
        return []
    
    result = tokenizer.analyze(title)
    result = [[morph.lex, morph.tag] for split in result for morph in split.morphs]  # (형태소, 품사) 리스트의 리스트
    return result


이건 토큰화 되었을 때, 토큰-tag 맵핑 json을 이용해서 플레이리스트에서 태그 10개 추출하는 것입니다. 여기 문제는 일단 무조건 10개 뽑는겁니다. 중복이 안되면 소용이 없다는게 제일 큰 문제네요ㅠㅠ

In [ ]:
def sort_tag(i, j, k):
  icm.extend(token['tags'][k]) #태그를 test에 추가해준다
  vocab = Counter(icm)  # tag중에 중복이 많은거 위주로 카운트한다
  vocab_size = 10 # 중복 많은거 중 10개까지, 근데 문제는 10개가 안되도 10개가 뽑힌다
  vocab = vocab.most_common(vocab_size) #중복 없이 추출된건 가장 왼쪽에 있는 거라서 기준이 없다
  for x in range(len(vocab)): #vocab 길이만큼 추가~
    test['tags'][i].append(vocab[x][0]) # 태그에 추가한다 extend로 하니까 음절단위로 끊겨서 append로 해야 하네요


In [ ]:
using_pos = ['NNG','SL','NNP', 'XR', 'VA'] #일반명사, 외국어, 고유명사, 어근, 형용사 만 사용할거입니다
rid=['노래','때', '좋', '모음', '곡',  '날', '야', '동', '같', '지',  '추천', '트', '요즘', '이', '없', '속', '스', '전', '을', '를', '로', '디', '래', '후', '라']


이건 각각의 플레이리스트 타이틀에서 토큰을 뽑는 방법입니다. usingpos로 품사를 정하고, rid로 stopword 불용어를 제거해서 나타낸 방법입니다. 만약 test_token, val_token쓰시면 이 부분은 안해도 됩니다만 원래는 해야겠죠

In [ ]:
test['plylst_title'] = re_sub(test['plylst_title'])
test.loc[:,'1'] = test['plylst_title'].map(lambda x: get_token(x, tokenizer))
test['2'] = test['1'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
test['3'] = test['2'].map(lambda x: list(filter(lambda x: not x[0] in rid, x)))
test['plylst_token'] = test['3'].map(lambda x: [tag[0] for tag in x])
test = test.drop(['1'], axis =1)
test = test.drop(['2'], axis =1)
test = test.drop(['3'], axis =1)

In [ ]:
val['plylst_title'] = re_sub(val['plylst_title'])
val.loc[:,'1'] = val['plylst_title'].map(lambda x: get_token(x, tokenizer))
val['2'] = val['1'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
val['3'] = val['2'].map(lambda x: list(filter(lambda x: not x[0] in rid, x)))
val['plylst_token'] = val['3'].map(lambda x: [tag[0] for tag in x])
val = val.drop(['1'], axis =1)
val = val.drop(['2'], axis =1)
val = val.drop(['3'], axis =1)

In [ ]:
val = val.drop(['1'], axis =1)
val = val.drop(['2'], axis =1)
val = val.drop(['3'], axis =1)

tqdm은 진행현황알려구

In [ ]:
for i in tqdm(range (10739)): #전체 테스트 개수에 대해서 // 나중에 분담해도 될듯?
  if len(test['plylst_token'][i]) !=0: #토큰이 1개라도 뽑히면
    icm = []
    for j in range(len(test['plylst_token'][i])): #토큰의 개수에 따라 
      for k in range(len(token)): #토큰_태그 개수만큼 34901
        if test['plylst_token'][i][j] == token["token"][k]: #이 토큰을 토큰_태그에서 찾아서 
          sort_tag(i, j, k)
